# Build an Agent
By themselves, language models can't take actions - they just output text. Agents are systems that use an LLM as a reasoning enginer to determine which actions to take and what the inputs to those actions should be. The results of those actions can then be fed back into the agent and it determine whether more actions are needed, or whether it is okay to finish.

In [1]:
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Accessing the variables
# os.getenv("OPENAI_API_KEY")
os.getenv("LANGCHAIN_TRACING_V2")
os.getenv("LANGCHAIN_API_KEY")
os.getenv("TAVILY_API_KEY")
os.getenv("OPENAI_API_KEY")


'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'

### Define tools
We first need to create the tools we want to use. We will use two tools: Tavily (to search online) and then a retriever over a local index we will create

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search.invoke("what is the latest projects related to ai agents?")


[{'url': 'https://www.businessinsider.com/project-astra-google-ai-agent-dancing-gemini-2024-5?op=1',
  'content': "Joy Malone via Getty Images. Project Astra is an experimental effort to reimagine what AI agents can be in the future. I got to test out the new AI technology at Google's I/O conference. I danced ..."},
 {'url': 'https://www.businessinsider.com/google-future-ai-agents-project-astra-2024-5?op=1',
  'content': 'Project Astra is a window into this future. A vertical stack of three evenly spaced horizontal lines. ... AI agents are what have the best chance at taking this technology from a nice-to-have to a ...'}]

### Retriever
We will also create a retriever over some data of our own.

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [4]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

Now that we have populated our index that we will do doing retrieval over, we can easily turn it into a tool (the format needed for an agent to properly use it)

In [5]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

### Tools
Now that we have created both, we can create a list of tools that we will use downstream.

In [10]:
tools = [search, retriever_tool]

### Using Language Models
Next, let's learn how to use a language model by to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools. 

In [12]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.

In [13]:
from langchain_core.messages import HumanMessage

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: Hello! How can I assist you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.

In [14]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in queens, ny?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Queens, NY'}, 'id': 'call_oQ9hcgD9FOUPXUxMn7x69sf1'}]


We can see that there's now no content, but there is a tool call! It wants us to call the Tavily Search tool.

This isn't calling that tool yet - it's just telling us to. In order to actually calll it, we'll want to create our agent.

### Create the agent
Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Now, we can initalize the agent with the LLM and the tools.

Note that we are passing in the model, not model_with_tools. That is because create_tool_calling_executor will call .bind_tools for us under the hood.

In [16]:
from langgraph.prebuilt import chat_agent_executor

agent_executor = chat_agent_executor.create_tool_calling_executor(model, tools)

### Run the agent
We can now run the agent on a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

In [17]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]


[HumanMessage(content='hi!', id='66a6f6e1-a5fa-4efd-81eb-0ede2d57e430'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 129, 'total_tokens': 139}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6fa8fec2-a7b0-492f-b2d1-3aaa1585ca99-0')]

Let's now try it out on an example where it should be invoking the retriever

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="how can langsmith help with testing?")]}
)
response["messages"]

Now let's try one where it needs to call the search tool:

In [19]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Queens, NY?")]}
)
response["messages"]


[HumanMessage(content='whats the weather in Queens, NY?', id='a8274f2e-cefb-49a4-974a-38f8a9c48d28'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_GnHlCkdPIPSelulf23wwypwN', 'function': {'arguments': '{"query":"weather in Queens, NY"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 136, 'total_tokens': 158}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6348ba76-6773-4ab4-8cc8-9ed6defc0ba8-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Queens, NY'}, 'id': 'call_GnHlCkdPIPSelulf23wwypwN'}]),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Queens Village\', \'region\': \'New York\', \'country\': \'USA United States of America\', \'lat\': 40.73, \'lon\': -73.75, \'tz_id\': \'America/New_York\', \'localtime_epoch\

### Streaming Messages
We've seen how the agent can be called with .invoke to get back a final response. If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Queens, NY?")]}
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_agHxP2CJ63wTGYeH8h1TFXXb', 'function': {'arguments': '{"query":"weather in Queens, NY"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 136, 'total_tokens': 158}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-08396204-98d0-401e-86d4-16976fd9e52a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Queens, NY'}, 'id': 'call_agHxP2CJ63wTGYeH8h1TFXXb'}])]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Queens Village\', \'region\': \'New York\', \'country\': \'USA United States of America\', \'lat\': 40.73, \'lon\': -73.75, \'tz_id\': \'America/New_York\', \'localtime_epoch\': 1716325318, \'localtime\': \'2024-05-21 17:01\

### Streaming tokens
In addition to streaming back messages, it is also useful to be streaming back tokens. We can do this with the .astream_events method.

In [18]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in Queens, NY?")]},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")


c:\Users\samin\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in Queens, NY'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Queens Village', 'region': 'New York', 'country': 'USA United States of America', 'lat': 40.73, 'lon': -73.75, 'tz_id': 'America/New_York', 'localtime_epoch': 1716325318, 'localtime': '2024-05-21 17:01'}, 'current': {'last_updated_epoch': 1716325200, 'last_updated': '2024-05-21 17:00', 'temp_c': 21.7, 'temp_f': 71.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 180, 'wind_dir': 'S', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 71, 'cloud': 75, 'feelslike_c': 21.7, 'feelslike_f': 71.1, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 6.0, 'gust_mph': 15.5, 'gust_kph': 24.9}}"}, {'url': 'https://www.lo

### Adding in memory
As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).

In [21]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer=memory
)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 131, 'total_tokens': 142}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2ea347b2-03f7-4d42-bb81-a3bcf4dedbf1-0')]}}
----


In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")


{'agent': {'messages': [AIMessage(content='Your name is Bob, as you mentioned earlier. How can I help you, Bob?', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 154, 'total_tokens': 173}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a87c43cf-a45d-4a4e-a82e-fd58007e4e99-0')]}}
----
